In [3]:
import sys
sys.path.append(r"C:\Users\nshre\OneDrive\Desktop\DeepVision\models")


In [4]:
from CSRNet import CSRNet


In [ ]:
import sys
import cv2
import torch
import numpy as np
import gradio as gr

# Add your CSRNet models folder path
sys.path.append(r"C:\Users\nshre\OneDrive\Desktop\DeepVision\models")

# Import CSRNet
from CSRNet import CSRNet

# Load model
model = CSRNet()
model.load_state_dict(torch.load(
    r"C:\Users\nshre\OneDrive\Desktop\DeepVision\csrnet_model.pth",
    map_location="cpu"
))
model.eval()

frame_skip = 0

def fast_heatmap(frame, density):
    dens = density.copy()
    dens -= dens.min()
    if dens.max() != 0:
        dens /= dens.max()

    dens = (dens * 255).astype("uint8")
    heat = cv2.applyColorMap(dens, cv2.COLORMAP_JET)
    heat = cv2.resize(heat, (frame.shape[1], frame.shape[0]))

    return cv2.addWeighted(frame, 0.6, heat, 0.4, 0)

def process_frame(frame):
    global frame_skip
    if frame is None:
        return None, 0

    frame_skip += 1
    if frame_skip % 2 != 0:
        return frame, 0

    # Convert RGB -> BGR for OpenCV
    bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    small = cv2.resize(bgr, (256, 256))

    tensor = torch.from_numpy(small.transpose(2, 0, 1)).float().unsqueeze(0) / 255.0

    with torch.no_grad():
        density_map = model(tensor)
        count = float(density_map.sum().item())

    density_np = density_map[0][0].cpu().numpy()
    result_bgr = fast_heatmap(bgr, density_np)

    cv2.putText(result_bgr, f"Count: {count:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Convert back to RGB for Gradio
    result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)
    return result_rgb, count

# ⭐ Gradio Interface with webcam streaming
demo = gr.Interface(
    fn=process_frame,
    inputs=gr.Image(sources=["webcam"], streaming=True, type="numpy"),  # correct live webcam
    outputs=[
        gr.Image(label="Processed Output"),
        gr.Number(label="People Count")
    ],
    title="CSRNet – Fast Live Crowd Monitor",
    description="Live webcam crowd counting (CPU optimized).",
    live=True  # enables continuous streaming
)

demo.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
